In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
jeopardy=pd.read_csv("jeopardy.csv")
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [32]:
jeopardy.shape

(19999, 12)

In [3]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [4]:
jeopardy.columns=jeopardy.columns.str.lower()

In [5]:
jeopardy.columns=jeopardy.columns.str.strip()

In [6]:
jeopardy.columns

Index(['show number', 'air date', 'round', 'category', 'value', 'question',
       'answer'],
      dtype='object')

In [7]:
import re
def normalize_text(text):
    text=text.lower()
    text=re.sub("[^A-Za-z0-9\s]","",text)
    return text

In [8]:
jeopardy["clean_question"]=jeopardy["question"].apply(normalize_text)
jeopardy["clean_answer"]=jeopardy["answer"].apply(normalize_text)

In [9]:
def normalize_value(text):
    text=re.sub("[^A-Za-z0-9]","",text)
    try :
        text=int(text)
    except Exception:
        text=0
    return text

In [10]:
jeopardy["clean_value"]=jeopardy["value"].apply(normalize_value)

In [11]:
jeopardy.head()

,show number,air date,round,category,value,question,answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona,200
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds,200
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams,200


In [12]:
jeopardy["air date"]=pd.to_datetime(jeopardy["air date"])

In [13]:
jeopardy.dtypes

show number                int64
air date          datetime64[ns]
round                     object
category                  object
value                     object
question                  object
answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object

In [14]:
def ansq(row):
    split_answer=row["clean_answer"].split(" ")
    split_question=row["clean_question"].split(" ")
    match_count=0
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer)==0:
        return 0
    for i in split_answer:
        if i in split_question:
            match_count+=1
    return match_count/len(split_answer)


In [15]:
answer_in_question=jeopardy.apply(ansq,axis=1)
answer_in_question

0        0.000000
1        0.000000
2        0.000000
3        0.000000
4        0.000000
5        0.000000
6        0.000000
7        0.000000
8        0.000000
9        0.333333
10       0.000000
11       0.000000
12       0.000000
13       0.000000
14       0.500000
15       0.000000
16       0.000000
17       0.000000
18       0.000000
19       0.000000
20       0.000000
21       0.000000
22       0.000000
23       0.000000
24       0.500000
25       0.000000
26       0.000000
27       0.000000
28       0.000000
29       0.000000
           ...   
19969    0.000000
19970    0.000000
19971    0.000000
19972    0.000000
19973    0.000000
19974    0.333333
19975    0.000000
19976    0.000000
19977    0.000000
19978    0.000000
19979    0.000000
19980    0.500000
19981    0.500000
19982    0.000000
19983    0.000000
19984    0.000000
19985    0.000000
19986    0.000000
19987    0.000000
19988    0.000000
19989    0.000000
19990    0.000000
19991    0.000000
19992    0.000000
19993    0

In [16]:
answer_in_question.mean()

0.06049325706933587

we can see that the above mean gives us 0.06 propotion . it means 6% of the questions have some part of answer in them.

In [21]:
question_overlap=[]
terms_used=set()
jeopardy=jeopardy.sort_values("air date")
for i,row in jeopardy.iterrows():
    split_question=row["clean_question"].split(" ")
    split_question=[j for j in split_question if len(j)>5]
    match_count=0
    for i in split_question:
        if i in terms_used:
            match_count+=1
    for word in split_question:
        terms_used.add(word)
    if len(split_question)>0:
        match_count=match_count/len(split_question)
    question_overlap.append(match_count*100)
jeopardy["question_overlap_percentage"]=question_overlap
print(jeopardy["question_overlap_percentage"].mean())    

68.77843800700857


about 70 percent of the times similar words are repeated in questions. from this we cannot say the same question is repeated in the show . but what we can say is it is good to once look at previous questions because words from the questions match with previous questions. And maybe we can predict what type of questions are more frequently asked in the show.

In [22]:
def more(row):
    if row["clean_value"]>800:
        value=1
    else:
        value=0
    return value

In [25]:
jeopardy["high_value"]=jeopardy.apply(more,axis=1)

In [28]:
def counts(word):
    low_count=0
    high_count=0
    for i,row in jeopardy.iterrows():
        split_question=row["clean_question"].split(" ")
        if word in split_question:
            if row["high_value"]==1:
                high_count+=1
            else:
                low_count+=1
    return high_count,low_count
observed_expected=[]
comparison_terms=list(terms_used)[:5]
for i in comparison_terms:
    observed_expected.append(counts(i))
observed_expected

[(3, 13), (1, 1), (0, 1), (1, 0), (3, 6)]

In [34]:
high_value_count=jeopardy[jeopardy["high_value"]==1].shape[0]
high_value_count

5734

In [36]:
low_value_count=jeopardy[jeopardy["high_value"]==0].shape[0]
low_value_count

14265

In [38]:
from scipy.stats import chisquare
chi_squared=[]
for i in observed_expected:
    total=sum(i)
    total_prop=total/jeopardy.shape[0]
    expected_h=total_prop*high_value_count
    expected_l=total_prop*low_value_count
    
    observed = np.array([i[0], i[1]])
    expected = np.array([expected_h, expected_l])
    chi_squared.append(chisquare(observed, expected))

chi_squared

[Power_divergenceResult(statistic=0.7701156281377792, pvalue=0.3801812842139476),
 Power_divergenceResult(statistic=0.4448774816612795, pvalue=0.5047776487545996),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=0.09564350170321084, pvalue=0.75712159875701)]